In [1]:
import tensorflow as tf
### prepare inputs
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
seq1=np.array(random.choices("ACGT",k=131072))
seq2=np.array(random.choices("ACGT",k=131072))
ohe=OneHotEncoder()
a1=ohe.fit_transform(seq1.reshape(-1,1)).toarray()
a2=ohe.fit_transform(seq2.reshape(-1,1)).toarray()
_input=np.concatenate((a1.reshape(-1,a1.shape[0],a1.shape[1]),a2.reshape(-1,a2.shape[0],a2.shape[1])),axis=0)
_input.shape

def initialize_uninitialized(sess):
    global_vars          = tf.global_variables()
    is_not_initialized   = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]

    #print [str(i.name) for i in not_initialized_vars] # only for testing
    if len(not_initialized_vars):
        sess.run(tf.variables_initializer(not_initialized_vars))

    list_of_variables = tf.global_variables()

    uninitialized_variables = sess.run(tf.report_uninitialized_variables(list_of_variables))

    if len(uninitialized_variables)>0:
        return uninitialized_variables
    else:
        return "all_variables_initialized"

## restore Basenji model

In [2]:
tf.reset_default_graph() 

# Add ops to save and restore all the variables.
#saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# cpu
#with tf.Session() as sess: 
# gpu
gpu_options=tf.GPUOptions(allow_growth=True)
sess= tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 
# pre-trained model from author
new_saver = tf.train.import_meta_graph('./model.tf.meta') 
new_saver.restore(sess,'./model.tf')
graph=tf.get_default_graph()
writer = tf.summary.FileWriter("./model_graph", graph=graph,filename_suffix='_pre-trained')



# get some input/output
# inputs=graph.get_tensor_by_name('inputs:0').eval(feed_dict={'inputs:0':_input,'is_training:0':False})
# output1=graph.get_tensor_by_name('preds:0').eval(feed_dict={'inputs:0':_input,'is_training:0':False})
# sess.run(graph.get_operation_by_name('preds'),feed_dict={'inputs:0':_input,'is_training:0':False})
# output2=graph.get_tensor_by_name('preds:0').eval(feed_dict={'inputs:0':_input,'is_training:0':False}) 
  

W0911 21:36:23.320344  2132 deprecation.py:323] From c:\users\yynst\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [3]:
# need to re-define training op, loss function, and optimizer for other tasks
tmp_op=graph.get_tensor_by_name('final/dense/BiasAdd:0')

with tf.variable_scope('transfer_learn_branch'):
    new_op_1=tf.layers.dense(inputs=tmp_op,units=500,name='new_op_1')
    new_op_2=tf.layers.batch_normalization(inputs=new_op_1,name='new_op_2')
    new_op_3=tf.layers.average_pooling1d(inputs=new_op_2,pool_size=(10),strides=3,padding='valid',name='new_op_3')
    new_op_4=tf.layers.flatten(inputs=new_op_3,name='new_op_4')
    new_op_5=tf.layers.dense(inputs=new_op_4,units=1,name='new_op_5')

with tf.variable_scope('criteria'):
    new_ph=tf.placeholder(tf.float32,shape=[None,1],name='new_target_op')
    loss=tf.reduce_mean(tf.square(tf.subtract(new_ph,new_op_5)),name='new_loss_op')
    opt=tf.train.AdamOptimizer(learning_rate=0.01,
                               name='new_adam').minimize(loss,
                                                         name='new_opt', 
                                                         var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='transfer_learn_branch')
                                                        )
# initialize un-initialized variables
initialize_uninitialized(sess)



W0911 21:36:24.606347  2132 deprecation.py:323] From <ipython-input-3-b19da775f596>:5: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0911 21:36:24.612348  2132 deprecation.py:506] From c:\users\yynst\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0911 21:36:24.854344  2132 deprecation.py:323] From <ipython-input-3-b19da775f596>:6: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should no

'all_variables_initialized'

In [6]:
# show new graph
new_graph_def=tf.graph_util.extract_sub_graph(sess.graph_def,['inputs','transfer_learn_branch/new_op_5/BiasAdd'])

writer = tf.summary.FileWriter("./model_graph", graph=sess.graph_def,filename_suffix='_new_op_only')

#writer = tf.summary.FileWriter("./model_graph", graph=graph,filename_suffix='_new_op')


W0911 21:38:42.115795  2132 deprecation.py:323] From <ipython-input-6-d82348401cc1>:2: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
W0911 21:38:42.209795  2132 writer.py:199] Passing a `GraphDef` to the SummaryWriter is deprecated. Pass a `Graph` object instead, such as `sess.graph`.


In [4]:
graph.get_operations()

[<tf.Operation 'global_step/Initializer/zeros' type=Const>,
 <tf.Operation 'global_step' type=VariableV2>,
 <tf.Operation 'global_step/Assign' type=Assign>,
 <tf.Operation 'global_step/read' type=Identity>,
 <tf.Operation 'inputs' type=Placeholder>,
 <tf.Operation 'targets' type=Placeholder>,
 <tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'is_training' type=Placeholder>,
 <tf.Operation 'cnn0/Relu' type=Relu>,
 <tf.Operation 'cnn0/conv1d/kernel/Initializer/random_uniform/shape' type=Const>,
 <tf.Operation 'cnn0/conv1d/kernel/Initializer/random_uniform/min' type=Const>,
 <tf.Operation 'cnn0/conv1d/kernel/Initializer/random_uniform/max' type=Const>,
 <tf.Operation 'cnn0/conv1d/kernel/Initializer/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'cnn0/conv1d/kernel/Initializer/random_uniform/sub' type=Sub>,
 <tf.Operation 'cnn0/conv1d/kernel/Initializer/random_uniform/mul' type=Mul>,
 <tf.Operation 'cnn0/conv1d/kernel/Initializer/random_uniform' type=Add>,
 

## get trainable vars

In [4]:
graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'cnn0/conv1d/kernel:0' shape=(22, 4, 312) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/gamma:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/beta:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn1/conv1d/kernel:0' shape=(1, 312, 368) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/gamma:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/beta:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn2/conv1d/kernel:0' shape=(6, 368, 435) dtype=float32_ref>,
 <tf.Variable 'cnn2/batch_normalization/gamma:0' shape=(435,) dtype=float32_ref>,
 <tf.Variable 'cnn2/batch_normalization/beta:0' shape=(435,) dtype=float32_ref>,
 <tf.Variable 'cnn3/conv1d/kernel:0' shape=(6, 435, 514) dtype=float32_ref>,
 <tf.Variable 'cnn3/batch_normalization/gamma:0' shape=(514,) dtype=float32_ref>,
 <tf.Variable 'cnn3/batch_normalization/beta:0' shape=(514,) dtype=float32_ref>,
 <tf.Variable 'cnn4/conv1d/kernel:0' shap

In [7]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='transfer_learn_branch')

[<tf.Variable 'transfer_learn_branch/new_op_1/kernel:0' shape=(4229, 500) dtype=float32_ref>,
 <tf.Variable 'transfer_learn_branch/new_op_1/bias:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'transfer_learn_branch/new_op_2/gamma:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'transfer_learn_branch/new_op_2/beta:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'transfer_learn_branch/new_op_5/kernel:0' shape=(158500, 1) dtype=float32_ref>,
 <tf.Variable 'transfer_learn_branch/new_op_5/bias:0' shape=(1,) dtype=float32_ref>]

In [8]:
tf.global_variables()

[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>,
 <tf.Variable 'cnn0/conv1d/kernel:0' shape=(22, 4, 312) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/gamma:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/beta:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/moving_mean:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/moving_variance:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn1/conv1d/kernel:0' shape=(1, 312, 368) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/gamma:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/beta:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/moving_mean:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/moving_variance:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn2/conv1d/kernel:0' shape=(6, 368, 435) dtype=float32_ref>,
 <tf.Variable 'cnn2/batch_n

## get all operations

In [4]:
for _ in graph.get_operations():
    print(_.name)

global_step/Initializer/zeros
global_step
global_step/Assign
global_step/read
inputs
targets
Placeholder
is_training
cnn0/Relu
cnn0/conv1d/kernel/Initializer/random_uniform/shape
cnn0/conv1d/kernel/Initializer/random_uniform/min
cnn0/conv1d/kernel/Initializer/random_uniform/max
cnn0/conv1d/kernel/Initializer/random_uniform/RandomUniform
cnn0/conv1d/kernel/Initializer/random_uniform/sub
cnn0/conv1d/kernel/Initializer/random_uniform/mul
cnn0/conv1d/kernel/Initializer/random_uniform
cnn0/conv1d/kernel
cnn0/conv1d/kernel/Assign
cnn0/conv1d/kernel/read
cnn0/conv1d/dilation_rate
cnn0/conv1d/conv1d/ExpandDims/dim
cnn0/conv1d/conv1d/ExpandDims
cnn0/conv1d/conv1d/ExpandDims_1/dim
cnn0/conv1d/conv1d/ExpandDims_1
cnn0/conv1d/conv1d/Conv2D
cnn0/conv1d/conv1d/Squeeze
cnn0/batch_normalization/gamma/Initializer/ones
cnn0/batch_normalization/gamma
cnn0/batch_normalization/gamma/Assign
cnn0/batch_normalization/gamma/read
cnn0/batch_normalization/beta/Initializer/zeros
cnn0/batch_normalization/beta
cnn0

## transfer learning (wip)

In [121]:
#### example
# Retrieve VGG inputs
self.x_plh = vgg_graph.get_tensor_by_name('input:0')

# Choose which node you want to connect your own graph
output_conv = vgg_graph.get_tensor_by_name('conv1_2:0')
# output_conv =vgg_graph.get_tensor_by_name('conv2_2:0')
# output_conv =vgg_graph.get_tensor_by_name('conv3_3:0')
# output_conv =vgg_graph.get_tensor_by_name('conv4_3:0')
# output_conv =vgg_graph.get_tensor_by_name('conv5_3:0')

# Stop the gradient for fine-tuning
# You can also see this as using the VGG model as a feature extractor only
output_conv_sg = tf.stop_gradient(output_conv) # It's an identity function

# Build further operations
output_conv_shape = output_conv_sg.get_shape().as_list()
W1 = tf.get_variable('W1', shape=[1, 1, output_conv_shape[3], 32], initializer=tf.random_normal_initializer(stddev=1e-1))
b1 = tf.get_variable('b1', shape=[32], initializer=tf.constant_initializer(0.1))
z1 = tf.nn.conv2d(output_conv_sg, W1, strides=[1, 1, 1, 1], padding='SAME') + b1
a = tf.nn.relu(z1)